## Preamble

Scope of the study: evaluate regeression model by training, validating and testing Linear regression, Random Forest and DNN. 


Material datasets downloaded with Material Project datasets (MPR) API. Datasets used has Silicon as its primary element. Read dataset as pandas framework. Used matplotlib and pandas plotting function to visulalize histogram distribution, features correlation.


Used energy above hull as the parameter of interest (POI). Nine features used to train ML. Applied filters to remove duplicate, undstable, NaN cbm and vbm.


**Importing packages from:**

*DNN --> tensorflow*

*Random Forest & Linear Regression --> sklearn*

*Plotting*

*MPR*

*Other packages*

In [ ]:
import keras as ks
import tensorflow as tf
from tensorflow import keras

#import tensorflow.contrib.slim as slim
import scipy
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, re
from collections import OrderedDict, defaultdict
from flatten_dict import flatten
import seaborn as sns

# material project datasets
from mp_api.client.mprester import MPRester 


**Extract Information**

- Define features

- Obtain Data from MPR (Material Project Reader)

    - Filter unstable entries, band gap, formation energy

- Put the data in pandas datafram 

- Prepare pd.DataFrame for ML

- Add volume and magnetization descriptor

In [ ]:
field = ["formation_energy_per_atom", "band_gap", "energy_per_atom","total_magnetization",
         "volume","composition","energy_above_hull" ,"nelements",'nsites',"formula_pretty","is_stable","density"#]
         ,"vbm","cbm"]

apiid = 'fpVY0sh8FhtuhcJizsr8DROWxQ4AiYYR'



with MPRester(apiid) as mpr:
    docs = mpr.materials.summary.search(elements=['Si'],#,'O'],
                                        #num_elements = (2,10),
                                        #band_gap=(0, 10),
                                        #energy_above_hull=(0,0.1),
                                        #formation_energy=(-20,5),
                                        fields=field,
                                        #is_stable=True
                                        )
flattened = [{
      k: v
      for k, v in flatten(doc.dict(), reducer="dot").items()
      if k != "fields_not_requested"
  } for doc in docs]

df = pd.DataFrame.from_records(flattened)
print('before removing duplicate size=',df.shape)
df.drop_duplicates('formula_pretty', keep='first', inplace=True)
df.dropna(subset=["cbm","vbm"],inplace=True) # filter cbm and vbm
df  = df[(df['formation_energy_per_atom']>-20) & (df['formation_energy_per_atom']<5)]
print('after removing duplicate size=',df.shape)

all_columns_inmpr = df.columns
df_mp = df.drop(columns=[col for col in df if col not in field])

df_mp['vpa'] = df['volume']/df['nsites']
df_mp['magmom_pa'] = df['total_magnetization'] / df['nsites']

df_mp.head()

**Step1:**

- Explore dataset and statistics

- Visualize features: histograms and correlation

In [ ]:
df_mp.head()

In [ ]:
df_mp.describe()

In [ ]:
excluded = ['energy_above_hull','volume','nsites','is_stable','formula_pretty','total_magnetization']
included = [c for c in df_mp.columns if c not in excluded]
df_corr = df_mp[included]

plt.figure(figsize=(8, 6))
sns.heatmap(df_corr.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation ")
plt.show()

In [ ]:
df_corr.hist(figsize=(12, 8), bins=30, color='skyblue', edgecolor='black')
plt.suptitle("Feature Distributions", fontsize=16)
plt.yscale('log')
plt.show()

# Linear Regression

Fit Linear Regression model:

- Define target and features

- Fit Linear Regression model

- Plot results

- Cross validate results

In [ ]:
y = df_mp['energy_above_hull'].values
excluded = ['energy_above_hull','volume','nsites','is_stable','formula_pretty','total_magnetization']
included = [c for c in df_mp.columns if c not in excluded]
X = df_mp[included].values

#print("Random Forest features are: {}".format(X))

In [ ]:
lr = LinearRegression()

lr.fit(X, y)

# get fit statistics
print('R2 = ' + str(round(lr.score(X, y), 3)))
print('RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=lr.predict(X))))

In [ ]:
plt.plot(lr.predict(X),y,'.')
plt.plot([0,7],[0,7])
plt.xlabel('predict')
plt.ylabel('true')
plt.title('Linear Regression Model: true vs predict')

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

# Use 10-fold cross validation (90% training, 10% test)
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)

# compute cross validation scores for random forest model
scores = cross_val_score(lr, X, y, scoring='neg_mean_squared_error',
                         cv=crossvalidation, n_jobs=1)
rmse_scores = [np.sqrt(abs(s)) for s in scores]

print('Cross-validation results:')
print('Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores))))

# Random Forest Regression

Fit Random Forest regression model:

- Define target and features

- Fit random forest model

- Plot the results

- Cross validate results

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=1)

rf.fit(X, y)
print('R2 = ' + str(round(rf.score(X, y), 3)))
print('RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=rf.predict(X))))

In [ ]:
plt.plot(rf.predict(X), y,'.')
plt.plot([0,7],[0,7])
plt.xlabel('predict')
plt.ylabel('true')
plt.title("Random Forest Regressor")

In [ ]:
scores = cross_val_score(rf, X, y, scoring='neg_mean_squared_error', cv=crossvalidation, n_jobs=1)

rmse_scores = [np.sqrt(abs(s)) for s in scores]
print('Cross-validation results:')
print('Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores))))

In [ ]:
importances = rf.feature_importances_
included = np.asarray(included)
indices = np.argsort(importances)[::-1]

plt.bar(included[indices],height=importances[indices])
plt.tick_params(axis='x', rotation=80)
plt.yscale('log')
plt.title('Random Forest Feature importance')

# DNN:

- Used same dataset features to train, validate and test DNN; used 70% to train, and 15% each to validate and test

- Used 14 dense layers with 1024 neurons (units)

- Activation function: ReLU (rectified linear units)

- Dropout 30% of neuorons to prevent overfitting

- Batch normalization (layer that normalizes input batch to its mean and std) for each dense layer

- early stopping applied to avoid overfitting

- Adam optimizer used with learning rate = 0.0003 (to avoid overfitting), loss function "mae"

In [ ]:
y = df_mp['energy_above_hull'].values
excluded = ['energy_above_hull','volume','nsites','is_stable','formula_pretty','total_magnetization']
included = [c for c in df_mp.columns if c not in excluded]
X = df_mp[included].values


from sklearn.model_selection import train_test_split
X_train, X_temp,y_train, y_temp = train_test_split(X,y, test_size=0.3, random_state=1234567)
X_val,   X_test,y_val,   y_test = train_test_split(X_temp,y_temp,test_size=0.5, random_state=1234567)

#X_train.shape, X_valid.shape, X.shape

In [ ]:
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=[9]),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1),
])



adam = keras.optimizers.Adam(learning_rate=0.0003)

model.compile(
    optimizer=adam,
    loss='mae',
)
print(model.summary())


early_stopping = keras.callbacks.EarlyStopping(
    patience = 5,
    min_delta = 0.001,
    restore_best_weights = True
)

In [ ]:
epochs = 40
batch = 256

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=batch,
    epochs=epochs,
    callbacks=[early_stopping]
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df['loss'][3:].plot()
history_df['val_loss'][3:].plot()
plt.legend(['loss','val_loss'])
plt.yscale('log')
#history_df['accuracy'].plot()
#history_df['val_accuracy'].plot()


In [ ]:
test_loss = model.evaluate(X_test, y_test)
#print(f"Test Accuracy: {test_acc}, Test loss: {test_loss}")
print('test loss: {test_loss}')

ydnn_pred = model.predict(X_test)
plt.plot(ydnn_pred,y_test,'o')
plt.plot([0,7],[0,7])
plt.xlabel("predict")
plt.ylabel("true")
plt.title("DNN energy above hull: true vs prediction")

# XGBoost with DNN

In [ ]:
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

# Load dataset
#data = load_diabetes()
#X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
xgb_model.fit(X_train, y_train)

# Get feature importance
important_features = xgb_model.feature_importances_
selected_features = X_train[:, important_features > 0.01]  # Filtering based on importance

# Define DNN model
dnn_model = Sequential([
    Dense(64, activation="relu", input_shape=(selected_features.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)  # Regression output
])

# Compile & train
dnn_model.compile(optimizer="adam", loss="mse")
dnn_model.fit(selected_features, y_train, epochs=10, batch_size=16)
